In [ ]:
import tensorflow as tf
import cv2
print(tf.__version__)
!python —-version
cv2.__version__

2.8.0
python3: can't open file '—-version': [Errno 2] No such file or directory


'4.1.2'

In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
if os.path.exists('/content/gdrive/MyDrive/data/car_image/'):
 !rm -fr '/content/gdrive/MyDrive/data/car_image/'
!mkdir -p '/content/gdrive/MyDrive/data/car_image/'
%cd '/content/gdrive/MyDrive/data/car_image/'
!ls -lia

/content/gdrive/MyDrive/data/car_image
total 0


In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 72752, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 72752 (delta 116), reused 224 (delta 91), pack-reused 72491
Receiving objects: 100% (72752/72752), 579.33 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (51426/51426), done.
Checking out files: 100% (3039/3039), done.


In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,615 kB of archives.
After this operation, 8,916 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu

In [ ]:
%cd '/content/gdrive/MyDrive/data/car_image/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/MyDrive/data/car_image/models/research


In [ ]:
import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/MyDrive/data/car_image/models"
sys.path.append("/content/gdrive/MyDrive/data/car_image/models/research/")

In [ ]:
%cd '/content/gdrive/MyDrive/data/car_image/models/research/'
!cp object_detection/packages/tf2/setup.py .
# . == 현재 디렉토리

/content/gdrive/MyDrive/data/car_image/models/research


In [ ]:
!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detect

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-05-11 04:09:20.809722: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0511 04:09:21.051425 140685854873472 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 4.02s
I0511 04:09:21.414235 140685854873472 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 4.02s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.54s
I0511 04:09:21.958091 140685854873472 test

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import pathlib
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
 tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)
  model_dir = pathlib.Path(model_dir)/"saved_model"
  model = tf.saved_model.load(str(model_dir))
  return model

In [ ]:
PATH_TO_LABELS = '/content/gdrive/MyDrive/data/car_image/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
use_display_name=True)

#model_name = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
#model_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8'
detection_model = load_model(model_name)
print(detection_model.signatures['serving_default'].inputs)
detection_model.signatures['serving_default'].output_dtypes
detection_model.signatures['serving_default'].output_shapes

346005504/345992335 [==============================] - 3s 0us/step
[<tf.Tensor 'input_tensor:0' shape=(1, None, None, 3) dtype=uint8>, <tf.Tensor 'unknown:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_0:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_1:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_2:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_3:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_4:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_5:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_6:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_7:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_8:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_9:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_10:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_11:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_12:0' shape=<unknown> dtype=resource>, <tf.Tensor 'unknown_13:0' shape=<unknown> dtype=reso

{'detection_boxes': TensorShape([1, 100, 4]),
 'detection_classes': TensorShape([1, 100]),
 'detection_scores': TensorShape([1, 100]),
 'num_detections': TensorShape([1])}

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
 
 # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
 
 # All outputs are batches tensors.
 # Convert to numpy arrays, and take index [0] to remove the batch dimension.
 # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
 
 # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

 # Handle models with masks:
  if 'detection_masks' in output_dict:
 # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
        output_dict['detection_masks'], output_dict['detection_boxes'],
          image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                    tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

In [ ]:
def show_inference_pics(model, image_path):
 
 # the array based representation of the image will be used later in order to prepare the
 # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
 # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
 # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=8)
 
  display(Image.fromarray(image_np))

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/gdrive/MyDrive/data/car_image/images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[]

In [ ]:
for image_path in TEST_IMAGE_PATHS: 
 show_inference_pics(detection_model, image_path)

Output hidden; open in https://colab.research.google.com to view.